In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from eipy.ei import EnsembleIntegration
import eipy.utils as ut
from eipy.additional_ensembles import MeanAggregation, CES
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import datasets
pd.set_option('display.max_columns', None)

In [2]:
# If data is multi-class, run a check on the allowable base and meta models.

base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(multi_class="auto", solver="lbfgs"),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}

In [3]:

"""
For filtering base predictors by whether or not they rely on heursitics for multiclass extension

natively_multi_class_predictors = ["XGBClassifier",
"BernoulliNB",
"DecisionTreeClassifier",
"ExtraTreeClassifier",
"GaussianNB",
"KNeighborsClassifier",
"LabelPropagation",
"LabelSpreading",
"LinearDiscriminantAnalysis",
"LinearSVC", #(setting multi_class=”crammer_singer”)
"LogisticRegression", #(setting multi_class=”multinomial”)
"LogisticRegressionCV", #(setting multi_class=”multinomial”)
"MLPClassifier",
"NearestCentroid",
"QuadraticDiscriminantAnalysis",
"RadiusNeighborsClassifier",
"RandomForestClassifier",
"RidgeClassifier",
"RidgeClassifierCV"]

base_predictors = {k : v for k,v in base_predictors.items() if str(v).split("(")[0] in natively_multi_class_predictors}
"""

'\nFor filtering base predictors by whether or not they rely on heursitics for multiclass extension\n\nnatively_multi_class_predictors = ["XGBClassifier",\n"BernoulliNB",\n"DecisionTreeClassifier",\n"ExtraTreeClassifier",\n"GaussianNB",\n"KNeighborsClassifier",\n"LabelPropagation",\n"LabelSpreading",\n"LinearDiscriminantAnalysis",\n"LinearSVC", #(setting multi_class=”crammer_singer”)\n"LogisticRegression", #(setting multi_class=”multinomial”)\n"LogisticRegressionCV", #(setting multi_class=”multinomial”)\n"MLPClassifier",\n"NearestCentroid",\n"QuadraticDiscriminantAnalysis",\n"RadiusNeighborsClassifier",\n"RandomForestClassifier",\n"RidgeClassifier",\n"RidgeClassifierCV"]\n\nbase_predictors = {k : v for k,v in base_predictors.items() if str(v).split("(")[0] in natively_multi_class_predictors}\n'

In [4]:
"""https://dev.pages.lis-lab.fr/scikit-multimodallearn/tutorial/auto_examples/combo/plot_combo_3_views_3_classes.html#
multi modal multi-class toy data generation"""

def generate_data(n_samples, lim):
    """Generate random data in a rectangle"""
    lim = np.array(lim)
    n_features = lim.shape[0]
    data = np.random.random((n_samples, n_features))
    data = (lim[:, 1]-lim[:, 0]) * data + lim[:, 0]
    return data
seed = 12
np.random.seed(seed)

n_samples = 300

modality_0 = np.concatenate((generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]])))

modality_1 = np.concatenate((generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]])))

modality_2 = np.concatenate((generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]])))

X = np.concatenate([modality_0,modality_1,modality_2], axis=1)

y = np.zeros(3*n_samples, dtype=np.int64)
y[n_samples:2*n_samples] = 1
y[2*n_samples:] = 2



In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

In [6]:
X_0_train, X_0_test = X_train[:, 0:2], X_test[:, 0:2]
X_1_train, X_1_test = X_train[:, 2:4], X_test[:, 2:4]
X_2_train, X_2_test = X_train[:, 4:], X_test[:, 4:]

In [7]:
data_train = {
                "Modality_0": X_0_train,
                "Modality_1": X_1_train,
                "Modality_2": X_2_train
                }

data_test = {
                "Modality_0": X_0_test,
                "Modality_1": X_1_test,
                "Modality_2": X_2_test
                }

In [8]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,
                        n_jobs=-1,
                        random_state=42,
                        project_name="toy",
                        model_building=True,
                        )


In [9]:
EI.train_base(X=data_train, y=y_train)

Training base predictors on None...
        
... for ensemble performance analysis...


Generating meta training data: |          |  0%

Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


...


... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


In [10]:
EI.base_summary

{'metrics': modality       Modality_0                                                    \
 base predictor       ADAB        DT        GB       KNN        LR       MLP   
 precision        0.525585  0.525310  0.554165  0.518227  0.565432  0.561388   
 recall           0.663835  0.525739  0.602771  0.565332  0.592944  0.629683   
 f1               0.533093  0.525519  0.566932  0.534460  0.575164  0.568177   
 
 modality                                               Modality_1            \
 base predictor        NB        RF       SVM       XGB       ADAB        DT   
 precision       0.560510  0.534797  0.544832  0.543818   0.441781  0.557991   
 recall          0.622572  0.572382  0.622702  0.563717   0.666667  0.552616   
 f1              0.569864  0.548096  0.556542  0.552061   0.531397  0.555141   
 
 modality                                                                    \
 base predictor        GB       KNN        LR       MLP        NB        RF   
 precision       0.555493  

In [11]:
EI.meta_training_data[0]

modality          Modality_0                                              \
base predictor          ADAB                               XGB             
sample                     0                                 0             
class                      0             1         2         0         1   
0               5.030195e-01  2.222401e-16  0.496980  0.484714  0.000684   
1               2.223461e-16  5.037648e-01  0.496235  0.000010  0.996938   
2               5.030195e-01  2.222401e-16  0.496980  0.188556  0.001075   
3               2.223461e-16  5.037648e-01  0.496235  0.000255  0.986222   
4               5.030195e-01  2.222401e-16  0.496980  0.973815  0.001691   
..                       ...           ...       ...       ...       ...   
571             2.223057e-16  5.034980e-01  0.496502  0.000772  0.029771   
572             2.223057e-16  5.034980e-01  0.496502  0.000891  0.451275   
573             2.223057e-16  5.034980e-01  0.496502  0.000244  0.930683   
574             2.223057e-16  5.034980e-01  0.496502  0.005836  0.573228   
575             5.033028e-01  2.222778e-16  0.496697  0.851118  0.001734   

modality                                                                       \
base predictor             DT              RF                    GB             
sample                      0               0                     0             
class                  2    0    1    2     0     1     2         0         1   
0               0.514602  1.0  0.0  0.0  0.43  0.00  0.57  0.409006  0.004771   
1               0.003051  0.0  1.0  0.0  0.00  0.94  0.06  0.009681  0.758107   
2               0.810369  1.0  0.0  0.0  0.43  0.07  0.50  0.081958  0.004527   
3               0.013523  0.0  1.0  0.0  0.00  0.98  0.02  0.005364  0.861940   
4               0.024494  1.0  0.0  0.0  0.89  0.00  0.11  0.785019  0.008433   
..                   ...  ...  ...  ...   ...   ...   ...       ...       ...   
571             0.969457  0.0  0.0  1.0  0.07  0.36  0.57  0.005927  0.712334   
572             0.547833  0.0  0.0  1.0  0.02  0.33  0.65  0.010632  0.836332   
573             0.069073  0.0  1.0  0.0  0.00  0.94  0.06  0.007694  0.788019   
574             0.420936  0.0  1.0  0.0  0.00  0.51  0.49  0.013155  0.831022   
575             0.147149  0.0  0.0  1.0  0.72  0.01  0.27  0.535095  0.009329   

modality                                                               \
base predictor            KNN                  LR                       
sample                      0                   0                       
class                  2    0    1    2         0         1         2   
0               0.586223  0.6  0.0  0.4  0.721130  0.031602  0.247268   
1               0.232212  0.0  1.0  0.0  0.137229  0.396308  0.466464   
2               0.913515  0.4  0.2  0.4  0.205876  0.245101  0.549022   
3               0.132696  0.0  1.0  0.0  0.030080  0.648603  0.321317   
4               0.206548  0.6  0.0  0.4  0.743277  0.020614  0.236109   
..                   ...  ...  ...  ...       ...       ...       ...   
571             0.281738  0.4  0.2  0.4  0.242135  0.294048  0.463817   
572             0.153036  0.0  0.6  0.4  0.029223  0.667539  0.303238   
573             0.204287  0.0  1.0  0.0  0.034983  0.590646  0.374371   
574             0.155824  0.0  0.6  0.4  0.134426  0.329729  0.535845   
575             0.455576  0.8  0.0  0.2  0.394456  0.129577  0.475967   

modality                                                                    \
base predictor        NB                           MLP                       
sample                 0                             0                       
class                  0         1         2         0         1         2   
0               0.805465  0.000146  0.194389  0.717909  0.006161  0.275930   
1               0.035621  0.526522  0.437858  0.098913  0.490241  0.410846   
2               0.152868  0.171413  0.675719  0.178818  0.280640  0.540543   
3

In [12]:
EI.train_meta(meta_predictors=base_predictors)

Analyzing ensembles: |██████████|100%
Training final meta models: |██████████|100%


In [13]:
EI.meta_summary["metrics"]

,ADAB,XGB,DT,RF,GB,KNN,LR,NB,MLP,SVM
precision,0.112500,0.379134,0.112500,0.222994,0.112500,0.355449,0.251544,0.222222,0.298480,0.333333
recall,0.333333,0.379443,0.333333,0.331134,0.333333,0.349724,0.373494,0.334986,0.299372,0.333334
f1,0.168224,0.379208,0.168224,0.263965,0.168224,0.325664,0.280447,0.264200,0.269546,0.326469


In [14]:
preferred_meta_model = EI.meta_summary["metrics"].loc["precision"].idxmax()
y_pred = EI.predict(X_dict=data_test, meta_model_key=preferred_meta_model)
y_pred = [np.argmax(np.array(y)) for y in y_pred]
y_pred

[2,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 1,
 0,
 1,
 2,
 2,
 0,
 1,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 1,
 0,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 1,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 1,
 2,
 0,
 2,
 1,
 0,
 0,
 0,
 2,
 1,
 2,
 1,
 2,
 0,
 1,
 2,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 1,
 1,
 0,
 2,
 2,
 1,
 1,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2]

In [15]:
y_test

array([1, 2, 1, 0, 0, 1, 1, 0, 1, 1, 2, 1, 1, 0, 2, 0, 0, 2, 1, 0, 2, 2,
       0, 1, 2, 0, 1, 1, 1, 2, 0, 0, 0, 2, 1, 0, 1, 0, 2, 0, 1, 1, 2, 0,
       2, 0, 2, 1, 0, 2, 0, 1, 0, 2, 0, 1, 1, 1, 2, 0, 1, 2, 1, 1, 0, 2,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 2, 2, 0, 0, 1, 2, 0, 1, 2, 2, 1, 2,
       2, 1, 2, 1, 0, 1, 1, 2, 1, 2, 2, 0, 0, 0, 0, 0, 2, 0, 1, 1, 0, 2,
       2, 1, 2, 2, 2, 2, 2, 0, 2, 1, 1, 1, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1,
       0, 0, 0, 1, 1, 2, 2, 0, 1, 2, 0, 2, 2, 0, 1, 0, 2, 1, 2, 2, 2, 1,
       1, 1, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 1, 0, 0, 2, 1, 1, 1, 2, 2,
       1, 0, 2, 1])

In [16]:
accuracy = sum([1*(y==y_hat)+0*(y!=y_hat) for y,y_hat in list(zip(y_test, y_pred))])/len(y_test)
accuracy

0.3388888888888889

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

X = np.concatenate([modality_0,modality_1,modality_2], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3, stratify=y)

model = LogisticRegression(multi_class='auto', solver='lbfgs', max_iter=1000)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9222222222222223

In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        60
           1       0.89      0.92      0.90        60
           2       0.94      0.85      0.89        60

    accuracy                           0.92       180
   macro avg       0.92      0.92      0.92       180
weighted avg       0.92      0.92      0.92       180



In [19]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

Modality_a = X[:, 0:2]
Modality_b = X[:, 2:4]

X_a_train, X_a_test, y_train, y_test = train_test_split(Modality_a, y, test_size=0.2, random_state=3, stratify=y)
X_b_train, X_b_test, _,_ = train_test_split(Modality_b, y, test_size=0.2, random_state=3, stratify=y)

In [20]:
iris_data_train = {
                "Modality_a": X_a_train,
                "Modality_b": X_b_train
                }

iris_data_test = {
                "Modality_a": X_a_test,
                "Modality_b": X_b_test
                }

In [21]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}

In [22]:
EI_iris = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,
                        n_jobs=-1,
                        random_state=0,
                        project_name="iris",
                        model_building=True,
                        )


In [23]:
for name, modality in iris_data_train.items():
    EI_iris.train_base(modality, y_train, modality_name=name)

Training base predictors on Modality_a...
        
... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on Modality_b...
        
... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


In [24]:
EI_iris.meta_training_data

[modality          Modality_a                                              \
 base predictor          ADAB                               XGB             
 sample                     0                                 0             
 class                      0         1             2         0         1   
 0               4.700114e-01  0.529988  3.032480e-07  0.062144  0.930889   
 1               3.618822e-07  0.498631  5.013683e-01  0.000280  0.996896   
 2               3.618822e-07  0.498631  5.013683e-01  0.001002  0.029516   
 3               3.618822e-07  0.498631  5.013683e-01  0.002396  0.023360   
 4               4.700114e-01  0.529988  3.032480e-07  0.995907  0.002983   
 ..                       ...       ...           ...       ...       ...   
 91              2.142084e-12  0.000414  9.995858e-01  0.000806  0.126304   
 92              1.406017e-05  0.524571  4.754154e-01  0.004578  0.968059   
 93              9.872006e-01  0.012759  4.027025e-05  0.997407  0.001570   

In [25]:
EI_iris.train_meta(meta_predictors=base_predictors)

Analyzing ensembles: |██████████|100%
Training final meta models: |██████████|100%


In [26]:
EI_iris.meta_summary["metrics"]

,ADAB,XGB,DT,RF,GB,KNN,LR,NB,MLP,SVM
precision,0.111111,0.333333,0.111111,0.333333,0.111111,0.333333,0.111111,0.222222,0.111111,0.222222
recall,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
f1,0.166667,0.325758,0.166667,0.309524,0.166667,0.325758,0.166667,0.240196,0.166667,0.264069


In [27]:
preferred_meta_model = EI_iris.meta_summary["metrics"].loc["precision"].idxmax()
y_pred_iris = EI_iris.predict(X_dict=iris_data_test, meta_model_key=preferred_meta_model)
y_pred_iris = [np.argmax(np.array(y)) for y in y_pred_iris]
y_pred_iris

[0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 0,
 1,
 2,
 2,
 0,
 0]

In [28]:
accuracy = sum([1*(y==y_hat)+0*(y!=y_hat) for y,y_hat in list(zip(y_test, y_pred_iris))])/len(y_test)
accuracy

0.2

In [29]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train,y_train)
y_pred = xgb_model.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
accuracy

XGBoostError: [19:33:13] /workspace/src/data/data.cc:501: Check failed: this->labels.Size() % this->num_row_ == 0 (120 vs. 0) : Incorrect size for labels.
Stack trace:
  [bt] (0) /home/opc/.venv/lib64/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x3581ea) [0x7fc7c6e991ea]
  [bt] (1) /home/opc/.venv/lib64/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x389457) [0x7fc7c6eca457]
  [bt] (2) /home/opc/.venv/lib64/python3.9/site-packages/xgboost/lib/libxgboost.so(+0x38a4b1) [0x7fc7c6ecb4b1]
  [bt] (3) /home/opc/.venv/lib64/python3.9/site-packages/xgboost/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0xb0) [0x7fc7c6c9f210]
  [bt] (4) /lib64/libffi.so.6(ffi_call_unix64+0x4c) [0x7fc8b46ba17e]
  [bt] (5) /lib64/libffi.so.6(ffi_call+0x36f) [0x7fc8b46b9b2f]
  [bt] (6) /usr/lib64/python3.9/lib-dynload/_ctypes.cpython-39-x86_64-linux-gnu.so(+0x1375d) [0x7fc8b48d075d]
  [bt] (7) /usr/lib64/python3.9/lib-dynload/_ctypes.cpython-39-x86_64-linux-gnu.so(+0x13db5) [0x7fc8b48d0db5]
  [bt] (8) 